# OOP dopravní podnik

V tomto notebooku jsou dvě třídy:

- __Linka__ obsahuje seznam zastávek
    - umí hledat cestu mezi dvěma zastávkami
- __DopravniPodnik__ je síť Linek (uvnitř jde o slovník {název: Linka})
    - umí zjistit přestupní stanice
    - umí hledat cestu s přestupy

Omezení: DopravniPodnik neumí najít potřebné přestupní stanice sám, je potřeba mu je "poradit" pomocí parametru.

Co si z toho vzít?

- DopravniPodnik používá metody Linky, nedělá všechnu práci sám.
- V případě chyby jsou vyhozeny výjimky s popisem, co je špatně.
- Jak vypadá kód včetně dokumentace.
    - Popis metody může být delší než samotná metoda. To je v pořádku.
    - Z dokumentace má být jasné, co metody potřebují za parametry, co vracejí, jak třídu používat.
    - Jeden příklad použití kódu vydá za tisíc slov (`>>>`).

Jak psát `"""docstringy"""` (Google Python Style Guide): https://google.github.io/styleguide/pyguide.html?showone=Comments#Comments

In [1]:
METRO_A = ["Depo Hostivař", "Skalka", "Strašnická", "Želivského", "Flora", "Jiřího z Poděbrad",
           "Náměstí Míru", "Muzeum", "Můstek", "Staroměstská", "Malostranská", "Hradčanská",
           "Dejvická", "Bořislavka", "Nádraží Veleslavín", "Petřiny", "Nemocnice Motol"]

METRO_B = ["Zličín", "Stodůlky", "Luka", "Lužiny", "Hůrka", "Nové Butovice", "Jinonice", "Radlická",
           "Smíchovské nádraží", "Anděl", "Karlovo náměstí", "Národní třída", "Můstek", "Náměstí Republiky",
           "Florenc", "Křižíkova", "Invalidovna", "Palmovka", "Českomoravská", "Vysočanská", "Kolbenova",
           "Hloubětín", "Rajská zahrada", "Černý Most"]

METRO_C = ["Letňany", "Prosek", "Střížkov", "Ládví", "Kobylisy", "Nádraží Holešovice", "Vltavská", "Florenc",
           "Hlavní nádraží", "Muzeum", "I. P. Pavlova", "Vyšehrad", "Pražského povstání", "Pankrác", "Budějovická",
           "Kačerov", "Roztyly", "Chodov", "Opatov", "Háje"]

TRAM_1 = ["Spojovací", "Kněžská luka", "Chmelnice", "Strážní", "Vozovna Žižkov", "Ohrada", "Krejcárek", "Palmovka",
          "Libeňský most", "Maniny", "Tusarova", "Pražská tržnice", "Vltavská", "Strossmayerovo náměstí", "Kamenická",
          "Letenské náměstí", "Korunovační", "Sparta", "Hradčanská", "Prašný most", "Vozovna Střešovice", "Sibeliova",
          "Ořechovka", "Baterie", "Vojenská nemocnice", "Větrník", "Petřiny", "Sídliště Petřiny"]

In [2]:
class Linka:
    """
    Linka mestske dopravy.
    
    Atributy:
        linka.zastavky - seznam nazvu zastavek
    
    Metody:
        delka() - pocet zastavek
        najdi_trasu() - najde trasu mezi dvema zastavkami
    
    """
    def __init__(self, zastavky):
        """
        Vytvori novou linku.
        
        Parametry:
            zastavky - seznam nazvu zastavek
        
        """
        self.zastavky = zastavky
    
    def delka(self):
        """Pocet zastavek"""
        return len(self.zastavky)
    
    def najdi_trasu(self, nastup, vystup):
        """
        Najdi trasu mezi dvema zastavkami
        
        Parametry:
            nastup - nazev prvni zastavky
            vystup - nazev posledni zastavky
        
        Vraci:
            seznam zastavek na trase - [nastup, ...mezistanice..., vystup]
        
        Vyjimky:
            ValueError - nastup nebo vystup neni stanice na dane lince
        
        """
        if nastup not in self.zastavky or vystup not in self.zastavky:
            raise ValueError("Linka neobsahuje nekterou z koncovych zastavek trasy.")
        
        i = self.zastavky.index(nastup)  # self.zastavky[i] == nastup
        j = self.zastavky.index(vystup)
        
        if i <= j:
            return self.zastavky[i:j+1]
        else:
            # vystupni zastavka je v seznamu zastavek jako prvni, trasu je potreba obratit
            trasa = self.zastavky[j:i+1]
            return list(reversed(trasa))

    def __repr__(self):
        return "<Linka {}>".format(self.zastavky)

linka_a = Linka(METRO_A)
linka_a

<Linka ['Depo Hostivař', 'Skalka', 'Strašnická', 'Želivského', 'Flora', 'Jiřího z Poděbrad', 'Náměstí Míru', 'Muzeum', 'Můstek', 'Staroměstská', 'Malostranská', 'Hradčanská', 'Dejvická', 'Bořislavka', 'Nádraží Veleslavín', 'Petřiny', 'Nemocnice Motol']>

In [3]:
print("Metro A má", linka_a.delka(), "zastávek.")

Metro A má 17 zastávek.


In [4]:
print(linka_a.najdi_trasu("Muzeum", "Dejvická"))
print(linka_a.najdi_trasu("Dejvická", "Muzeum"))

['Muzeum', 'Můstek', 'Staroměstská', 'Malostranská', 'Hradčanská', 'Dejvická']
['Dejvická', 'Hradčanská', 'Malostranská', 'Staroměstská', 'Můstek', 'Muzeum']


In [5]:
class DopravniPodnik:
    """
    Systém dopravních linek s přestupy
    
    Metody:
        pridej_linku()
        vypis_prehled()
        prestupni_stanice() - najde stanice, ktere jsou na vice linkach
        najdi_zastavku() - najde linky, na kterych se nachazi zastavka
        najdi_trasu() - najde trasu na nektere lince
        najdi_trasu_s_prestupy() - najde cestu pres vice linek
    
    """
    def __init__(self):
        """
        Vytvori prazdnou sit linek
        
        Pro vytvoreni site pouzijte metodu pridej_linku().
        
        """
        self.linky = {}
    
    def pridej_linku(self, nazev, linka):
        """
        Prida linku do dopravni site
        
        Parametry:
            nazev - jmeno linky
            linka - objekt tridy Linka (popis zastavek na lince)
        
        """
        if not isinstance(linka, Linka):
            raise TypeError("Parametr linka musi byt objekt tridy Linka.")
        
        self.linky[nazev] = linka
    
    def vypis_prehled(self):
        """Vypis vsech linek a jejich nazvu"""
        for nazev, linka in self.linky.items():
            print(nazev)
            for zastavka in linka.zastavky:
                print("  -", zastavka)
            print()  # prazdny radek
    
    def prestupni_stanice(self):
        """Vrati seznam stanic, ktere jsou na vic nez jedne lince"""
        pocitadlo = {}  # jmeno zastavky -> pocet linek, na kterych se nachazi
        
        for _, linka in self.linky.items():
            for zastavka in linka.zastavky:
                if zastavka not in pocitadlo:
                    pocitadlo[zastavka] = 1
                else:
                    pocitadlo[zastavka] += 1
        
        prestupni = []
        
        for zastavka, pocet_linek in pocitadlo.items():
            if pocet_linek > 1:
                prestupni.append(zastavka)

        return prestupni
    
    def najdi_zastavku(self, zastavka):
        """
        Vrati seznam linek pro danou zastavku
        
        Parametry:
            zastavka - nazev hledane zastavky
        
        Vraci:
            seznam nazvu linek, na kterych se zastavka nachazi
        
        Vyjimky:
            ValueError - zastavka neni na zadne lince
        
        """
        vystup = []
        for nazev_linky, linka in self.linky.items():
            if zastavka in linka.zastavky:
                vystup.append(nazev_linky)
            
        if not vystup:
            raise ValueError("Zastavka {} neni na zadne lince.".format(zastavka))
        
        return vystup
    
    def najdi_trasu(self, nastup, vystup):
        """
        Zjisti trasu mezi zastavkami nastup, vystup na jedne lince
        
        Priklad:
            >>> dp.najdi_trasu("nastup", "vystup")
            (["linka 1"], ["nastup", "mezistanice 1", "mezistanice 2", "vystup"])
        
        Parametry:
            nastup: nazev prvni stanice (musi byt na teze lince)
            vystup: nazev posledni stanice (musi byt na teze lince)
        
        Vraci:
            dvojice (nazev_linky, seznam_zastavek)
        
        Vyjimky:
            ValueError - zastavky nelezi na stejne lince, zastavky nejsou vubec zname
        
        """
        
        # TODO: funkce sama najde vhodne prestupy (-> hledani cesty v grafu)
        
        linky_s_nastupem = self.najdi_zastavku(nastup)
        linky_s_vystupem = self.najdi_zastavku(vystup)
        for linka in linky_s_nastupem:
            if linka in linky_s_vystupem:
                return linka, self.linky[linka].najdi_trasu(nastup, vystup)
        
        raise ValueError("Zastavky {} a {} nemaji spolecnou linku.".format(nastup, vystup))
    
    def najdi_trasu_s_prestupy(self, nastup, vystup, prestupy):
        """
        Zjisti trasu mezi zastavkami nastup, vystup se zadanymi prestupy
        
        Najde se trasa nastup -> prestupy[0] -> ... -> prestupy[-1] -> vystup.
        
        Priklad:
            >>> dp.najdi_trasu_s_prestupy("nastup", "vystup", ["prestupni"])
            (["linka 1", "linka 2"],
             [["nastup", "prestupni"], ["prestupni", "vystup"]])
        
        Parametry:
            nastup: nazev prvni stanice
            vystup: nazev posledni stanice
            prestupy: seznam nazvu stanic, kde se ma zmenit linka
        
        Vraci:
            dvojice (seznam_linek, seznam_zastavek_na_linkach)
        
        Vyjimky:
            ValueError - linky se neprotinaji nebo nektera zastavka neexistuje
        
        """
        trasa_po_castech = []
        pouzite_linky = []
        
        linka, cast_trasy = self.najdi_trasu(nastup, prestupy[0])
        trasa_po_castech.append(cast_trasy)
        pouzite_linky.append(linka)

        for i in range(len(prestupy)-1):
            linka, cast_trasy = self.najdi_trasu(prestupy[i], prestupy[i+1])
            trasa_po_castech.append(cast_trasy)
            pouzite_linky.append(linka)
        
        linka, cast_trasy = self.najdi_trasu(prestupy[-1], vystup)
        trasa_po_castech.append(cast_trasy)
        pouzite_linky.append(linka)
        
        return pouzite_linky, trasa_po_castech

pid = DopravniPodnik()  # Pražská Integrovaná Doprava
pid.pridej_linku("Metro A", linka_a)
pid.vypis_prehled()

pid.pridej_linku("Metro B", Linka(METRO_B))
pid.pridej_linku("Metro C", Linka(METRO_C))
pid.pridej_linku("Tram č. 1", Linka(TRAM_1))
# pid.vypis_prehled()

Metro A
  - Depo Hostivař
  - Skalka
  - Strašnická
  - Želivského
  - Flora
  - Jiřího z Poděbrad
  - Náměstí Míru
  - Muzeum
  - Můstek
  - Staroměstská
  - Malostranská
  - Hradčanská
  - Dejvická
  - Bořislavka
  - Nádraží Veleslavín
  - Petřiny
  - Nemocnice Motol



In [6]:
pid.prestupni_stanice()

['Florenc',
 'Muzeum',
 'Hradčanská',
 'Můstek',
 'Palmovka',
 'Petřiny',
 'Vltavská']

In [7]:
pid.najdi_trasu("Hradčanská", "Vltavská")

('Tram č. 1',
 ['Hradčanská',
  'Sparta',
  'Korunovační',
  'Letenské náměstí',
  'Kamenická',
  'Strossmayerovo náměstí',
  'Vltavská'])

In [8]:
pid.najdi_trasu("Hradčanská", "Háje")

ValueError: Zastavky Hradčanská a Háje nemaji spolecnou linku.

In [9]:
pid.najdi_trasu_s_prestupy("Hradčanská", "Háje", ["Muzeum"])

(['Metro A', 'Metro C'],
 [['Hradčanská', 'Malostranská', 'Staroměstská', 'Můstek', 'Muzeum'],
  ['Muzeum',
   'I. P. Pavlova',
   'Vyšehrad',
   'Pražského povstání',
   'Pankrác',
   'Budějovická',
   'Kačerov',
   'Roztyly',
   'Chodov',
   'Opatov',
   'Háje']])

In [10]:
pid.najdi_trasu_s_prestupy("Hradčanská", "Háje", ["Můstek"])

ValueError: Zastavky Můstek a Háje nemaji spolecnou linku.

In [11]:
pid.najdi_trasu_s_prestupy("Pražská tržnice", "Jiřího z Poděbrad", ["Vltavská", "Muzeum"])

(['Tram č. 1', 'Metro C', 'Metro A'],
 [['Pražská tržnice', 'Vltavská'],
  ['Vltavská', 'Florenc', 'Hlavní nádraží', 'Muzeum'],
  ['Muzeum', 'Náměstí Míru', 'Jiřího z Poděbrad']])

## Bonus

In [12]:
# LinkaRychla dedi z tridy Linka

class LinkaRychla(Linka):
    """
    Rychlejsi implementace tridy Linka.
    
    """
    def __init__(self, zastavky):
        super().__init__(zastavky)  # volani konstruktoru, tj. __init__ v puvodni tride Linka
        
        self._cache_pro_trasy = {}  # (zacatek,konec) -> trasa
        
        # Slovnik self._cache_pro_trasy je implementacni detail, uzivatele tridy se o nej nemuseji zajimat.
        # V nove verzi tridy mozna rychle vyhledavani udelame jinak, nebo zjistime, ze neni potreba, chce moc pameti...
        # Protoze o tomto slovniku vi pouze metody v teto tride, muzeme ho snadno odstranit, aniz by to nekoho mrzelo.
        # Take dokumentace je lepsi, pokud neobsahuje zbytecne detaily, ale jen to, co uzivatele zajima -
        # v nasem pripade uzivatele zajima jak ma pouzivat Linka.najdi_trasu(), jak to funguje vevnitr je podruzne.
        
        # Seznam self.zastavky nema jmeno s podtrzitkem a je zdokumentovan, takze ocekavame, ze do nej
        # budou uzivatele sami koukat. (V Pythonu neni zvykem uplne vse balit do metod typu get_zastavky() jako v Jave.)

    # Metoda s podtrzitkem, nase pomocna vec, neocekavame, ze by ji mel volat nekdo "zvenci".
    def _zapamatuj_trasu(self, nastup, vystup, trasa):
        """Uloz trasu do cache"""
        # cache vypada jako {("nastup", "vystup"): ["nastup", "mezistanice", "vystup"], ...}
        self._cache_pro_trasy[(nastup, vystup)] = trasa
    
    def _najdi_zapamatovanou_trasu(self, nastup, vystup):
        """Vrat trasu z cache, jinak None"""
        if (nastup, vystup) in self._cache_pro_trasy:
            return self._cache_pro_trasy[(nastup, vystup)]
        else:
            return None
    
    # metoda delka() se podedi z tridy Linka, neni potreba ji menit
    
    def najdi_trasu(self, nastup, vystup):
        # mame uz trasu v cache?
        zapamatovana_trasa = self._najdi_zapamatovanou_trasu(nastup, vystup)
        if zapamatovana_trasa is not None:
            return zapamatovana_trasa
        
        # zjistime trasu pomoci normalni metody, kterou uz mame hotovou ve tride Linka
        trasa = super().najdi_trasu(nastup, vystup)  # volani metody Linka.najdi_trasu() se stejnym "self"
        
        # ulozime vysledek do cache
        self._zapamatuj_trasu(nastup, vystup, trasa)
        return trasa


    
linka_tram_1 = Linka(TRAM_1)
rychla_linka_tram_1 = LinkaRychla(TRAM_1)

%timeit linka_tram_1.najdi_trasu("Větrník", "Sídliště Petřiny")
%timeit rychla_linka_tram_1.najdi_trasu("Větrník", "Sídliště Petřiny")

100000 loops, best of 3: 3.16 µs per loop
The slowest run took 18.00 times longer than the fastest. This could mean that an intermediate result is being cached 
1000000 loops, best of 3: 648 ns per loop
